In [1]:
%load_ext autoreload
%autoreload 2
from CSE142L.notebook import *
from notebook import *
from cfiddle import *
# if get something about NUMEXPR_MAX_THREADS being set incorrectly, don't worry.  It's not a problem.

#KEY include Namebox.ipynb

<div style=" font-size: 300% !important;
    margin-top: 1.5em;
    margin-bottom: 1.5em;
    font-weight: bold;
    line-height: 1.0;
    text-align:center;">Lab 4: The Memory Hierarchy (Part II) -- Demos</div>


# Image Stabilization

# Instruction Latency

## Let's measure the latency of an mov instruction

Check the dependences, this doesn't really measure the latency of the mov.  It measures the add...

Can we clean this up?  Maybe turn it into a template based on type?


In [ ]:
def measure_latency(source, function, divisor, *argc, **kwargs):
    
    compiled =build(source, arg_map(OPTIMIZE="-O0"))
    display(compiled[0].source(function))
    display(compiled[0].cfg(function))

    with local_execution():
        run_data = run(compiled, function, *argc, **kwargs)
    df = PE_calc(run_data.as_df())
    df["Cycles/iteration"] = df["cycles"]/df["count"]
    df["Cycles/inst"] = df["cycles"]/df["count"]/(divisor+0.0)
    display(df[["Cycles/iteration", "Cycles/inst"]])

In [ ]:
measure_latency(code(r"""
#include"cfiddle.hpp"
#include<vector>
#include<cstdint>

extern "C"
uint64_t wide_1(register uint64_t count) {
    register uint64_t k =0;
    register uint64_t n =0;
    start_measurement();
	for(register uint64_t i = 0; i < count; i++) {
		n = k;
	}
    end_measurement();
    return n;
}
"""), "wide_1", 1, arg_map(count=10000000))

## Let's measure the latency of an add instruction

In [ ]:
measure_latency(code(r"""
#include"cfiddle.hpp"
#include<vector>
#include<cstdint>

extern "C"
uint64_t wide_1(register uint64_t count) {
	register uint64_t j =0;
    start_measurement();
	for(register uint64_t i = 0; i < count; i++) {
		j = j+1;
		j = j+1;       
		j = j+1;       
		j = j+1;       
		j = j+1;
	}
    end_measurement();
    return j;
}
"""), "wide_1", 5, arg_map(count=10000000))

## Adding `double`

In [ ]:
measure_latency(code(r"""
#include"cfiddle.hpp"
#include<vector>
#include<cstdint>

extern "C"
double wide_1(register uint64_t count) {
	register double j =0;
    register double k =1.0;
    start_measurement();
	for(register uint64_t i = 0; i < count; i++) {
		j = j+k;
		j = j+k;       
		j = j+k;       
		j = j+k;       
		j = j+k;
	}
    end_measurement();
    return j;
}
"""), "wide_1", 5.0, arg_map(count=10000000))

In [ ]:
with local_execution():
    wide_1_run = run(wide_1, "wide_1", arg_map(count=10000000))
df = PE_calc(wide_1_run.as_df())
df["Cycles/iteration"] = df["cycles"]/df["count"]
df["Cycles/inst"] = df["cycles"]/df["count"]/5.0
display(df[["Cycles/iteration", "Cycles/inst"]])

## `float` add

In [ ]:
wide_1 =build(code(r"""
#include"cfiddle.hpp"
#include<vector>
#include<cstdint>

extern "C"
float wide_1(register uint64_t count) {
	register float j =0;
    register float k =1.0;
    start_measurement();
	for(register uint64_t i = 0; i < count; i++) {
		j = j+k;
		j = j+k;       
		j = j+k;       
		j = j+k;       
		j = j+k;
	}
    end_measurement();
    return j;
}
"""), arg_map(OPTIMIZE="-O0"))

display(wide_1[0].source("wide_1"))
display(wide_1[0].cfg("wide_1"))

In [ ]:
with local_execution():
    wide_1_run = run(wide_1, "wide_1", arg_map(count=10000000))
df = PE_calc(wide_1_run.as_df())
df["Cycles/iteration"] = df["cycles"]/df["count"]
df["Cycles/inst"] = df["cycles"]/df["count"]/5.0
display(df[["Cycles/iteration", "Cycles/inst"]])

## Floating Point Divide

In [ ]:
wide_1 =build(code(r"""
#include"cfiddle.hpp"
#include<vector>
#include<cstdint>

extern "C"
float wide_1(register uint64_t count) {
	register float j =0;
    register float k =1.0;
    start_measurement();
	for(register uint64_t i = 0; i < count; i++) {
		j = j/k;
		j = j/k;       
		j = j/k;       
		j = j/k;       
		j = j/k;
	}
    end_measurement();
    return j;
}
"""), arg_map(OPTIMIZE="-O0"))
display(wide_1[0].source("wide_1"))

display(wide_1[0].cfg("wide_1"))

In [ ]:
with local_execution():
    wide_1_run = run(wide_1, "wide_1", arg_map(count=10000000))
df = PE_calc(wide_1_run.as_df())
df["Cycles/iteration"] = df["cycles"]/df["count"]
df["Cycles/inst"] = df["cycles"]/df["count"]/5.0
display(df[["Cycles/iteration", "Cycles/inst"]])

## Integer Divide

In [ ]:
wide_1 =build(code(r"""
#include"cfiddle.hpp"
#include<vector>
#include<cstdint>

extern "C"
uint64_t wide_1(register uint64_t count) {
	register uint64_t  j =(uint64_t)-1;
    register uint64_t  k =2;
    start_measurement();
	for(register uint64_t i = 0; i < count; i++) {
		j = j/k;
		j = j/k;       
		j = j/k;       
		j = j/k;      
		j = j/k;
	}
    end_measurement();
    return j;
}
"""), arg_map(OPTIMIZE="-O0"))
display(wide_1[0].source("wide_1"))
display(wide_1[0].cfg("wide_1"))

In [ ]:
with local_execution():
    wide_1_run = run(wide_1, "wide_1", arg_map(count=10000000))
df = PE_calc(wide_1_run.as_df())
df["Cycles/iteration"] = df["cycles"]/df["count"]
df["Cycles/inst"] = df["cycles"]/df["count"]/5.0
display(df[["Cycles/iteration", "Cycles/inst"]])

## Parallel Divides

In [ ]:
wide_1 =build(code(r"""
#include"cfiddle.hpp"
#include<vector>
#include<cstdint>

extern "C"
float wide_1(register uint64_t count) {
	register float j =10.0000;
	register float jj =12.0000;
    register float k =1.0;
    start_measurement();
	for(register uint64_t i = 0; i < count; i++) {
		j = j/k;
		j = j/k;       
		j = j/k;       
		j = j/k;       
		j = j/k;

		jj = jj/k;
		jj = jj/k;       
		jj = jj/k;       
		jj = jj/k;       
		jj = jj/k;
    }
    end_measurement();
    return j + jj;
}
"""), arg_map(OPTIMIZE="-O0"))
display(wide_1[0].cfg("wide_1"))
display(wide_1[0].source("wide_1"))
with local_execution():
    wide_1_run = run(wide_1, "wide_1", arg_map(count=10000000))
df = PE_calc(wide_1_run.as_df())
df["Cycles/iteration"] = df["cycles"]/df["count"]
df["Cycles/inst"] = df["cycles"]/df["count"]/10.0
display(df[["Cycles/iteration", "Cycles/inst"]])

In [ ]:
display(div2.cfg)
df = render_csv("div2.csv")
df["Cycles/iteration"] = df["cycles"]/df["size"]
df["Cycles/op"] = df["cycles"]/df["size"]/5.0
display(df[["Cycles/iteration", "Cycles/inst", "CPI"]])

In [ ]:
measure_latency(code(r"""
#include"cfiddle.hpp"
#include<vector>
#include<cstdint>

extern "C"
double wide_1(uint64_t count, float arg1) {
	register double j =0;
	register double jj = 8324.93242;
	register double jjj = 83324.93242;
    register double k = arg1 + 7.3;
    
    start_measurement();
	for(unsigned i = 0; i < count; i++) {
		j = j/k;
		j = j/k;       
		j = j/k;       
		j = j/k;       
		j = j/k;

        jj = jj/k;
		jj = jj/k;       
		jj = jj/k;       
		jj = jj/k;       
		jj = jj/k;

        jjj = jjj/k;
		jjj = jjj/k;       
		jjj = jjj/k;       
		jjj = jjj/k;       
		jjj = jjj/k;
	}
    end_measurement();

	return j + jj + jjj;
}
"""), "wide_1", 16, arg_map(count=10000000, arg1=1.0))

In [ ]:
display(div3.cfg)
df = render_csv("div3.csv")
df["Cycles/iteration"] = df["cycles"]/df["size"]
df["Cycles/op"] = df["cycles"]/df["size"]/5.0
display(df[["Cycles/iteration", "Cycles/inst", "CPI"]])

In [ ]:
measure_latency(code(r"""
#include"cfiddle.hpp"
#include<vector>
#include<cstdint>

extern "C"
double wide_1(uint64_t count, float arg1) {
	register double j =0;
	register double jj = 8324.93242;
	register double jjj = 83324.93242;
	register double jjjj = 83324.93242;
    register double k = arg1 + 7.3;
    
    start_measurement();
	for(unsigned i = 0; i < count; i++) {
		j = j/k;
		j = j/k;       
		j = j/k;       
		j = j/k;       
		j = j/k;

        jj = jj/k;
		jj = jj/k;       
		jj = jj/k;       
		jj = jj/k;       
		jj = jj/k;

        jjj = jjj/k;
		jjj = jjj/k;       
		jjj = jjj/k;       
		jjj = jjj/k;       
		jjj = jjj/k;
        
        jjjj = jjjj/k;
		jjjj = jjjj/k;       
		jjjj = jjjj/k;       
		jjjj = jjjj/k;       
		jjjj = jjjj/k;
	}
    end_measurement();

	return j + jj + jjj+jjjj;
}
"""), "wide_1", 16, arg_map(count=10000000, arg1=1.0))


In [ ]:
display(div4.cfg)
df = render_csv("div4.csv")
df["Cycles/iteration"] = df["cycles"]/df["size"]
df["Cycles/op"] = df["cycles"]/df["size"]/5.0
display(df[["Cycles/iteration", "Cycles/inst", "CPI"]])

In [ ]:
measure_latency(code(r"""
#include"cfiddle.hpp"
#include<vector>
#include<cstdint>

extern "C"
double wide_1(uint64_t count, float arg1) {
	register double j =0;
	register double jj = 8324.93242;
	register double jjj = 83324.93242;
	register double jjjj = 83324.93242;
	register double jjjjj = 83324.93242;
    register double k = arg1 + 7.3;
    
    start_measurement();
	for(unsigned i = 0; i < count; i++) {
		j = j/k;
		j = j/k;       
		j = j/k;       
		j = j/k;       
		j = j/k;

        jj = jj/k;
		jj = jj/k;       
		jj = jj/k;       
		jj = jj/k;       
		jj = jj/k;

        jjj = jjj/k;
		jjj = jjj/k;       
		jjj = jjj/k;       
		jjj = jjj/k;       
		jjj = jjj/k;
        
        jjjj = jjjj/k;
		jjjj = jjjj/k;       
		jjjj = jjjj/k;       
		jjjj = jjjj/k;       
		jjjj = jjjj/k;

        jjjjj = jjjjj/k;
		jjjjj = jjjjj/k;       
		jjjjj = jjjjj/k;       
		jjjjj = jjjjj/k;       
		jjjjj = jjjjj/k;

	}
    end_measurement();

	return j + jj + jjj+jjjj+jjjjj;
}
"""), "wide_1", 16, arg_map(count=10000000, arg1=1.0))



# Floating point

1. Numerical instability -- sort and add
2. FMA speedup
3. Gallery of weird results.

# Vectors

## Floats

In [ ]:
cfgs=[]
asm=[]
files=[]

In [ ]:
os.environ["OPENMP"]="yes"
for opts in ["-O0", "-O1 -fno-openmp", "-O3 -fno-unroll-loops", "-O3 -march=skylake -fno-unroll-loops"]:
    filename = f"vsum-{opts.replace(' ','')}"
    fiddle("inst_lat.cpp", code="""
    #include"function_map.hpp"
    #include<cstdint>
    #include<cmath>

    extern "C"
    void __attribute__((noinline)) vsum(register float *a, register float *b, register float * c, register uint64_t n) {
    #ifdef _OPENMP
    #pragma omp simd
    #endif
        for(unsigned int i = 0; i < n; i++) {
            c[i] = a[i] + b[i];
        }
    }

    extern "C"
    uint64_t* wide_1(uint64_t * data, uint64_t size, uint64_t arg1) {
        auto a = new float[size];
        auto b = new float[size];
        auto c = new float[size];
        vsum(a,b,c, size);
        data[0] = a[4];
        return data;
    }
    FUNCTION(one_array_1arg, wide_1);
    """,
           function=["wide_1"], name=filename, analyze=False, opt=opts, run=["perf_count"], 
               cmdline=f"--size 10000000", 
               perf_cmdline="--stat-set PE.cfg --MHz 3500")
    cfgs.append(do_cfg("build/inst_lat.so", symbol="vsum"))
    asm.append(do_render_code("build/inst_lat.s", lang="gas", show="vsum"))
    files.append(f"{filename}.csv")

In [ ]:
display(asm[0]) # Nothing

In [ ]:
display(cfgs[1]) # Normal opt

In [ ]:
display(cfgs[2]) # OpenMP SIMD

In [ ]:
display(cfgs[3]) #OPenMP SIMD AVX

In [ ]:

df=render_csv(files)
df["file"] = files
df["GHz"] = 1/df["CT"]/1e9
df["CT_ns"] = df["CT"]*1e9
df["What we should get"]= df["IC"]*df["CPI"]
df[["file","IC", "CPI", "CT_ns", "ET",  "GHz", "What we should get"]]


## Integers


In [ ]:
icfgs=[]
iasm=[]
ifiles=[]

os.environ["OPENMP"]="yes"
for opts in ["-O0", "-O1 -fno-openmp", "-O3 -fno-unroll-loops", "-O3 -march=skylake -fno-unroll-loops"]:
    filename = f"vsum-int-{opts.replace(' ','')}"
    fiddle("inst_lat.cpp", code="""
    #include"function_map.hpp"
    #include<cstdint>
    #include<cmath>

    extern "C"
    void __attribute__((noinline)) vsum(register int *a, register int *b, register  * c, register size_t n) {
    #ifdef _OPENMP
    #pragma omp simd
    #endif
        for(unsigned int i = 0; i < n; i++) {
            c[i] = a[i] + b[i];
        }
    }

    extern "C"
    uint64_t* wide_1(uint64_t * data, uint64_t size, uint64_t arg1) {
        auto a = new int[size];
        auto b = new int[size];
        auto c = new int[size];
        vsum(a,b,c, size);
        data[0] = a[4];
        return data;
    }
    FUNCTION(one_array_1arg, wide_1);
    """,
           function=["wide_1"], name=filename, analyze=False, opt=opts, run=["perf_count"], 
               cmdline=f"--size 10000000", 
               perf_cmdline="--stat-set PE.cfg --MHz 3500")
    icfgs.append(do_cfg("build/inst_lat.so", symbol="vsum"))
    iasm.append(do_render_code("build/inst_lat.s", lang="gas", show="vsum"))
    ifiles.append(f"{filename}.csv")

In [ ]:
display(iasm[0]) # Nothing

In [ ]:
display(icfgs[1]) # Normal opt

In [ ]:
display(icfgs[2]) # Normal opt

In [ ]:
display(icfgs[3]) # Normal opt

In [ ]:

df=render_csv(ifiles)
df["file"] = ifiles
df["GHz"] = 1/df["CT"]/1e9
df["CT_ns"] = df["CT"]*1e9
df[["file","IC", "CPI", "CT_ns", "ET",  "GHz"]]

In [8]:
build(code(r""" aoeu"""))

  0%|          | 0/1 [00:00<?, ?it/s]

CFiddleException: CFiddle encountered an error (call `enable_debug()` for full details):
Build command failed:

make -R -f /cse142L/cfiddle/src/cfiddle/resources/make/cfiddle.make MORE_INCLUDES=-I/cse142L/CSE141pp-Tool-Moneta/moneta/ BUILD=.cfiddle/builds/build/53c056c7a79560388b34cea1fa6a8ad8_6e980da47a73eb849df27d1005288d07 CFIDDLE_INCLUDE=/cse142L/cfiddle/src/cfiddle/resources/include CFIDDLE_VPATH=.cfiddle/builds/anonymous_code:.cfiddle/builds/build/53c056c7a79560388b34cea1fa6a8ad8_6e980da47a73eb849df27d1005288d07 TARGET=x86_64-linux-gnu .cfiddle/builds/build/53c056c7a79560388b34cea1fa6a8ad8_6e980da47a73eb849df27d1005288d07/6e980da47a73eb849df27d1005288d07.so

g++ -c -Wall -Werror  -g3 -fPIC  -I. -I/cse142L/cfiddle/src/cfiddle/resources/include -I/usr/local/include  -I/cse142L/CSE141pp-Tool-Moneta/moneta/  -MMD -save-temps=obj -std=gnu++11  .cfiddle/builds/anonymous_code/6e980da47a73eb849df27d1005288d07.cpp -o .cfiddle/builds/build/53c056c7a79560388b34cea1fa6a8ad8_6e980da47a73eb849